In [35]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
import os

In [36]:
LEARNING_RATE = 0.001
BATCH_SIZE = 4096
N_EPOCHS = 40
dataDirectory = 'dataSet'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device = ", device)

Device =  cuda


In [37]:
if not os.path.exists(dataDirectory):
    os.makedirs(dataDirectory)

downLoad = False
if not os.path.exists(os.path.join(dataDirectory, 'MNIST')):
    downLoad = True
trainData = datasets.MNIST(
    root = dataDirectory,
    train = True,
    transform = ToTensor(),
    download=downLoad
)
testData = datasets.MNIST(
    root = dataDirectory,
    train = False,
    transform = ToTensor(),
    download=downLoad
)

In [38]:
print("train data : ", trainData.data.shape)
print("test data : ", testData.data.shape)

train data :  torch.Size([60000, 28, 28])
test data :  torch.Size([10000, 28, 28])


In [39]:
loaders = {
    'trainLoader' : DataLoader(
        dataset=trainData, 
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=1
    )
    ,
    'testLoader' : DataLoader(
        dataset=testData, 
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=1
    )
}
loaders

{'trainLoader': <torch.utils.data.dataloader.DataLoader at 0x1b7ca773940>,
 'testLoader': <torch.utils.data.dataloader.DataLoader at 0x1b7ca773ac0>}

In [40]:
def train(
        model : nn.Module,
        optimizer : opt.Adam,
        lossFunction : nn.CrossEntropyLoss
):
        model.train()
        for idx, (data, target) in enumerate(loaders['trainLoader']):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = lossFunction(output, target)
            loss.backward()
            optimizer.step()


@torch.no_grad()
def test(
        model : nn.Module,
        lossFunction : nn.CrossEntropyLoss
):
    model.eval()
    testLoss = 0
    correct = 0
    for data, target in loaders['testLoader']:
        data, target = data.to(device), target.to(device)
        output = model(data)
        testLoss += lossFunction(output, target).item()
        pred = output.argmax(dim = 1, keepdim = True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    testLoss /= len(loaders['testLoader'].dataset)
    print("Accuracy : ", 100*correct / len(loaders['testLoader'].dataset))

In [41]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 300)
        self.fc2 = nn.Linear(300, 50)
        self.fc3 = nn.Linear(50, 10)
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)

In [42]:
mlp = MLP().to(device=device)
optimizer = opt.Adam(mlp.parameters(), lr=LEARNING_RATE)
lossFunction = nn.CrossEntropyLoss()

In [43]:
for _ in range(N_EPOCHS):
    train(
        model=mlp,
        optimizer=optimizer,
        lossFunction=lossFunction
    )
test(
    model=mlp,
    lossFunction=lossFunction
)   
    

Accuracy :  96.24
